This is a starting point for a more complex data analysis - but let's lay some groundwork.

**Mission:** Predict Moneylines from current standings and starting pitcher ERA

**Complexity:** Small

**Backtested:** No

**Chances of beating the books:** Low


## Imports

### Pip installs (for if not running locally)

In [35]:
!pip install mlb-statsapi
!pip install scikit-learn
!pip install seaborn


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: C:\Users\Aaron\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: C:\Users\Aaron\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: C:\Users\Aaron\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


### Imports

In [1]:
import mlbstatsapi
import requests

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings, pprint, random
warnings.filterwarnings('ignore')

# Set up plotting style
plt.style.use('default')
sns.set_palette('husl')

#when looking at a dataFrame, show all columns
pd.set_option('display.max_columns', None)

## Data Collection

Collect current MLB team standings, historical game results, and starting pitcher ERA data to build our moneyline prediction features.

In [41]:
def get_all_gamePks(season, start_month=1, end_month=12):
    season = requests.get('https://statsapi.mlb.com/api/v1/schedule', params={
        'startDate': f'{season}-{start_month}-01',
        'endDate': f'{season}-{end_month}-31',
        'sportId': 1,
        'gameType': 'R',  # Regular season games
        'language': 'en',
        'hydrate': 'game(boxscore)',
    }).json()
    return [game['gamePk'] for date in season['dates'] for game in date['games']]


twothree_gamepks = get_all_gamePks(2023)  # Example usage to fetch all game Pks for the 2023 season
twothree_gamepks[100]

718688

In [42]:
def get_line_score(game_pk):
    """
    Fetches the line score for a given game using its gamePk.
    """
    url = f'https://statsapi.mlb.com/api/v1/game/{game_pk}/linescore'
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error fetching line score for game {game_pk}: {response.status_code}")
        return None

def get_box_score(game_pk):
    """
    Fetches the box score for a given game using its gamePk.
    """
    url = f'https://statsapi.mlb.com/api/v1/game/{game_pk}/boxscore'
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error fetching box score for game {game_pk}: {response.status_code}")
        return None
    
box = get_box_score(twothree_gamepks[100]) # Example usage to fetch box score for the first gamePk
line = get_line_score(twothree_gamepks[100]) # Example usage to fetch line score for the first gamePk
# pprint.pprint(box['teams']['home'])

In [43]:
def get_player_name(box, player_id, home=True):
    """
    Extracts player name from player ID using the box score data structure.
    
    Parameters:
    - box: Box score dictionary from MLB API
    - player_id: Player ID (integer or string like 'ID502043')
    - home: Boolean, True for home team, False for away team
    
    Returns:
    - Player's full name or None if not found
    """
    if not box or 'teams' not in box:
        return None
    
    # Handle different ID formats
    if isinstance(player_id, int):
        player_key = f'ID{player_id}'
    elif isinstance(player_id, str) and not player_id.startswith('ID'):
        player_key = f'ID{player_id}'
    else:
        player_key = player_id
    
    # Get the appropriate team's player data
    team_key = 'home' if home else 'away'
    if team_key not in box['teams'] or 'players' not in box['teams'][team_key]:
        return None
    
    players = box['teams'][team_key]['players']
    
    # Look up the player
    if player_key in players:
        player_data = players[player_key]
        if 'person' in player_data and 'fullName' in player_data['person']:
            return player_data['person']['fullName']
    
    return None

# Example usage:
player_name = get_player_name(box, 502043, home=True)  # Kyle Gibson
print(player_name)
player_name = get_player_name(box, 'ID543056', home=True)  # Danny Coulombe
player_name

Kyle Gibson


'Danny Coulombe'

In [55]:
def get_team_postgame_record(box, home=True):
    """
    Extracts the pregame record of the team from the box score.
    """
    if 'teams' in box:
        team = box['teams']['home'] if home else box['teams']['away']
        return {
            'wins': team['team']['record']['wins'],
            'losses': team['team']['record']['losses'],
        }
    return None

def get_starter_pregame_stats(box, home=True, stats=['earnedRuns', 'hits', 'baseOnBalls', 'homeRuns', 'outs'], inc_player_name=False):
    """
    Extracts the starting pitcher's ERA before the game from the box score.

    possible stats to take: ['gamesPlayed', 'gamesStarted', 'flyOuts', 'groundOuts', 'airOuts', 'runs', 'doubles', 'triples',
     'homeRuns', 'strikeOuts', 'baseOnBalls', 'intentionalWalks', 'hits', 'hitByPitch', 'atBats', 'obp', 'caughtStealing', 'stolenBases',
      'stolenBasePercentage', 'numberOfPitches', 'era', 'inningsPitched', 'wins', 'losses', 'saves', 'saveOpportunities', 'holds',
       'blownSaves', 'earnedRuns', 'whip', 'battersFaced', 'outs', 'gamesPitched', 'completeGames', 'shutouts', 'pitchesThrown',
        'balls', 'strikes', 'strikePercentage', 'hitBatsmen', 'balks', 'wildPitches', 'pickoffs', 'groundOutsToAirouts', 'rbi',
         'winPercentage', 'pitchesPerInning', 'gamesFinished', 'strikeoutWalkRatio', 'strikeoutsPer9Inn', 'walksPer9Inn', 'hitsPer9Inn',
          'runsScoredPer9', 'homeRunsPer9', 'inheritedRunners', 'inheritedRunnersScored', 'catchersInterference', 'sacBunts', 'sacFlies',
           'passedBall', 'popOuts', 'lineOuts']

    """
    if 'teams' in box and 'pitchers' in box['teams']['home']:
        pitchers = box['teams']['home']['pitchers'] if home else box['teams']['away']['pitchers']
        starter = box['teams']['home']['players'][f'ID{pitchers[0]}'] if home else box['teams']['away']['players'][f'ID{pitchers[0]}']
        postgame = starter['seasonStats']['pitching']
        game = starter['stats']['pitching'] 

        # Create the stats dictionary
        stats_dict = {stat: postgame.get(stat, 0) - game.get(stat, 0) for stat in stats}
        
        # Add player name
        if inc_player_name:
            stats_dict['playername'] = get_player_name(box, pitchers[0], home=home)
        
        return stats_dict
    return None

def get_batters_pregrame_stats(box, home=True, stats=['hits', 'totalBases', 'homeRuns', 'plateAppearances', 'baseOnBalls']):
    """
    Extracts the pregame stats of a batter from the box score.

    dict_keys(['summary', 'gamesPlayed', 'flyOuts', 'groundOuts', 'airOuts', 'runs', 'doubles', 'triples', 'homeRuns',
    'strikeOuts', 'baseOnBalls', 'intentionalWalks', 'hits', 'hitByPitch', 'atBats', 'caughtStealing', 'stolenBases',
      'stolenBasePercentage', 'groundIntoDoublePlay', 'groundIntoTriplePlay', 'plateAppearances', 'totalBases', 'rbi',
        'leftOnBase', 'sacBunts', 'sacFlies', 'catchersInterference', 'pickoffs', 'atBatsPerHomeRun', 'popOuts', 'lineOuts']
    """
    if 'teams' in box and 'battingOrder' in box['teams']['home']:
        batters = box['teams']['home']['battingOrder'] if home else box['teams']['away']['battingOrder']
        # print(batters)
        batter_stats = list()
        batter_sums = {
            stat: 0 for stat in stats
        }
        for batter_id in batters:
            batter = box['teams']['home']['players'][f'ID{batter_id}'] if home else box['teams']['away']['players'][f'ID{batter_id}']
            postgame = batter['seasonStats']['batting']
            game = batter['stats']['batting']
            batter_stats.append({
                stat: postgame.get(stat, 0) - game.get(stat, 0) for stat in stats
            })
            batter_stats[-1]['playername'] = get_player_name(box, batter_id, home=home)
            for stat in stats:
                if stat != 'playername':
                    batter_sums[stat] += batter_stats[-1][stat]
        batter_stats.append(batter_sums)
        return batter_stats
    return None

get_batters_pregrame_stats(box, stats=['hits', 'totalBases', 'homeRuns', 'plateAppearances', 'baseOnBalls', 'playername'])

[{'hits': 6,
  'totalBases': 13,
  'homeRuns': 2,
  'plateAppearances': 28,
  'baseOnBalls': 4,
  'playername': 'Cedric Mullins'},
 {'hits': 8,
  'totalBases': 11,
  'homeRuns': 1,
  'plateAppearances': 29,
  'baseOnBalls': 2,
  'playername': 'Adley Rutschman'},
 {'hits': 2,
  'totalBases': 2,
  'homeRuns': 0,
  'plateAppearances': 8,
  'baseOnBalls': 0,
  'playername': 'Ryan McKenna'},
 {'hits': 7,
  'totalBases': 16,
  'homeRuns': 2,
  'plateAppearances': 28,
  'baseOnBalls': 2,
  'playername': 'Ryan Mountcastle'},
 {'hits': 2,
  'totalBases': 5,
  'homeRuns': 1,
  'plateAppearances': 22,
  'baseOnBalls': 6,
  'playername': 'Gunnar Henderson'},
 {'hits': 3,
  'totalBases': 7,
  'homeRuns': 1,
  'plateAppearances': 18,
  'baseOnBalls': 0,
  'playername': 'Ramón Urías'},
 {'hits': 6,
  'totalBases': 12,
  'homeRuns': 1,
  'plateAppearances': 19,
  'baseOnBalls': 2,
  'playername': 'Adam Frazier'},
 {'hits': 7,
  'totalBases': 13,
  'homeRuns': 1,
  'plateAppearances': 21,
  'baseOnBall

In [56]:
def get_final_score(line):
    """
    Extracts the final score from the line score.
    """
    if 'teams' in line and 'home' in line['teams'] and 'away' in line['teams']:
        home_runs = line['teams']['home']['runs']
        away_runs = line['teams']['away']['runs']
        return {
            'home_runs': home_runs,
            'away_runs': away_runs,
            'home_win': home_runs > away_runs
        }
    return None

In [57]:
line.keys()

dict_keys(['copyright', 'currentInning', 'currentInningOrdinal', 'inningState', 'inningHalf', 'isTopInning', 'scheduledInnings', 'innings', 'teams', 'defense', 'offense', 'balls', 'strikes', 'outs'])

In [58]:
LOG_EXPONENT = 10
import math

def get_team_elo(team_wins, team_losses, log_exp=LOG_EXPONENT):
    """
    Calculates the Elo rating for a team based on its wins and losses.
    """
    games_played = team_wins + team_losses
    if team_wins + team_losses == 0:
        return 1500  # Default Elo rating if no games played
    return 1500 + (team_wins / games_played) * (math.log((games_played + log_exp) /log_exp) + 1) * 100 # Simplified Elo calculation

def get_starter_averages(starter_stats):
    """
    Calculates the per inning stats for a starting pitcher.
    """
    if not starter_stats or starter_stats.get('outs', 0) == 0:
        return {stat: None for stat in starter_stats}
    total_innings = starter_stats.get('outs', 0) / 3
    return {stat: starter_stats.get(stat, 0) / total_innings * 9 for stat in starter_stats if stat not in ['outs', 'gamesPlayed', 'gamesStarted']}


# TODO: perhaps a non-weighted average would be sharper here, but for now...
def get_batting_team_averages(batter_stats):
    """
    Calculates the per game stats for a batter.
    """
    if not batter_stats or batter_stats.get('plateAppearances', 0) == 0:
        return {stat: None for stat in batter_stats}
    plateAppearances = batter_stats.get('plateAppearances', 1)  # Avoid division by zero
    return {stat: batter_stats.get(stat, 0) / plateAppearances for stat in batter_stats if stat not in ['gamesPlayed', 'battingOrder']}

In [59]:
BATTING_STATS = ['hits', 'totalBases', 'homeRuns', 'plateAppearances', 'baseOnBalls', 'playername']
PITCHING_STATS = ['earnedRuns', 'hits', 'baseOnBalls', 'homeRuns', 'outs']

def get_dataframe(game_pks):
    """
    Fetches box scores and line scores for a list of game Pks and returns a DataFrame.
    """
    data = []
    for game_pk in game_pks:
        box = get_box_score(game_pk)
        line = get_line_score(game_pk)
        if box and line:
            home_team_record = get_team_postgame_record(box, home=True)
            away_team_record = get_team_postgame_record(box, home=False)
            home_starter_stats = get_starter_pregame_stats(box, home=True, stats=PITCHING_STATS, inc_player_name=True)
            away_starter_stats = get_starter_pregame_stats(box, home=False, stats=PITCHING_STATS, inc_player_name=True)
            home_batters_stats = get_batters_pregrame_stats(box, home=True, stats=BATTING_STATS)
            away_batters_stats = get_batters_pregrame_stats(box, home=False, stats=BATTING_STATS)

            final_score = get_final_score(line)

            if final_score['home_win']:
                home_team_record['wins'] -= 1
                away_team_record['losses'] -= 1
            else:
                home_team_record['losses'] -= 1
                away_team_record['wins'] -= 1

            data.append({
                'gamePk': game_pk,
                'home_wins': home_team_record['wins'],
                'home_losses': home_team_record['losses'],
                'home_winpercentage': home_team_record['wins'] / (home_team_record['wins'] + home_team_record['losses']) if (home_team_record['wins'] + home_team_record['losses']) > 0 else 0,
                'home_elo': get_team_elo(home_team_record['wins'], home_team_record['losses']),
                'away_elo': get_team_elo(away_team_record['wins'], away_team_record['losses']),
                'away_wins': away_team_record['wins'],
                'away_losses': away_team_record['losses'],
                'away_winpercentage': away_team_record['wins'] / (away_team_record['wins'] + away_team_record['losses']) if (away_team_record['wins'] + away_team_record['losses']) > 0 else 0,
                'away_runs': final_score['away_runs'],
                'home_win': final_score['home_win'],
                **{f'home_sp_{stat}': home_starter_stats.get(stat, None) for stat in PITCHING_STATS + ['playername']},
                **{f'away_sp_{stat}': away_starter_stats.get(stat, None) for stat in PITCHING_STATS + ['playername']},
                **{f'home_{i}_{stat}': home_batters_stats[i].get(stat, None) for stat in BATTING_STATS for i in range(len(home_batters_stats) - 1)},
                **{f'away_{i}_{stat}': away_batters_stats[i].get(stat, None) for stat in BATTING_STATS for i in range(len(away_batters_stats) - 1)}
            })
            if (len(data) % 100) == 5:
                print(f"Processed {len(data)} games...")
                print(data[-1]['home_0_playername'], data[-1]['away_0_playername'], data[-1]['home_sp_playername'], data[-1]['away_sp_playername'])
    
    return pd.DataFrame(data)

twothree_df = get_dataframe(twothree_gamepks)  # Example usage to fetch DataFrame for the 2023 season


Processed 5 games...
Taylor Walls Zach McKinstry Shane McClanahan Eduardo Rodriguez
Processed 105 games...
Trea Turner Jonathan India Zack Wheeler Hunter Greene
Processed 205 games...
Andrew Benintendi Cedric Mullins Mike Clevinger Tyler Wells
Processed 305 games...
Taylor Ward Bobby Witt Jr. Shohei Ohtani Taylor Clarke
Processed 405 games...
Zack Short Austin Hays Matthew Boyd Grayson Rodriguez
Processed 505 games...
Brandon Nimmo Charlie Blackmon Tylor Megill Austin Gomber
Processed 605 games...
Christian Yelich Bobby Witt Jr. Adrian Houser Zack Greinke
Processed 705 games...
Lane Thomas Zach McKinstry Josiah Gray Joey Wentz
Processed 805 games...
Steven Kwan Lars Nootbaar Hunter Gaddis Jordan Montgomery
Processed 905 games...
Marcus Semien Jarred Kelenic Nathan Eovaldi Bryce Miller
Processed 1005 games...
Jonathan Schoop Ronald Acuña Jr. Mason Englert Charlie Morton
Processed 1105 games...
Donovan Solano Rob Refsnyder Pablo López James Paxton
Processed 1205 games...
Marcus Semien Ma

In [60]:
# Save dataframe to CSV
twothree_df.to_csv('twothree_df.csv', index=False)  # Save DataFrame to CSV file

### Marcel Projections - turning them from values to ratios

This is mostly a premade dataset for us, we just have to run these couple lines to allow us to join datasets more easily later

In [25]:
hitter_projections = pd.read_csv("twothree_marcel_hitters.csv")
pitcher_projections = pd.read_csv("twothree_marcel_pitchers.csv")


In [26]:
MARCEL_HITTER_LINEAR_STATS = ['R', 'H', '2B', '3B', 'HR', 'RBI', 'SB', 'CS', 'BB', 'SO', 'TB']
MARCEL_PITCHER_LINEAR_STATS = ['H', 'R', 'ER', 'HR', 'BB', 'IBB', 'HBP', 'SO', 'WP']


hitter_projections['TB'] = (hitter_projections['H'] + hitter_projections['2B'] + (2 * hitter_projections['3B']) + (3 * hitter_projections['HR']))

for hitting_stat in MARCEL_HITTER_LINEAR_STATS:
    hitter_projections[hitting_stat + "/pa"] = hitter_projections[hitting_stat] / hitter_projections['PA']  # Convert to per game

for pitching_stat in MARCEL_PITCHER_LINEAR_STATS:
    pitcher_projections[pitching_stat + "/ip"] = pitcher_projections[pitching_stat] / pitcher_projections['IP']  # Convert to per 9 innings


In [27]:
hitter_projections.to_csv("twothree_marcel_hitters.csv", index=False)
pitcher_projections.to_csv("twothree_marcel_pitchers.csv", index=False)

## Feature Engineering

Now let's create features for our moneyline prediction model using team standings and pitcher data.

### Import Data (if stored locally)
prevents the need to run Data Collection Section, which could take 40 minutes or so

In [57]:
# Get it back from CSV
twothree_df = pd.read_csv('twothree_df.csv')  # Load DataFrame from CSV file
hitter_projections = pd.read_csv("twothree_marcel_hitters.csv")
pitcher_projections = pd.read_csv("twothree_marcel_pitchers.csv")

### Old Approach - use a generic Bayesian Multiplier for each player

In [88]:


bayesian = {
    'earnedRuns': 9,
    'sp_hits': 17,
    'sp_baseOnBalls': 8,
    'homeRuns': 3,
    'outs': 54,
    'wh': 25,  # Weighted average for WHIP
    
    #Batting stats
    'hits': 14,
    'totalBases': 23,
    'homeRuns': 1,
    'plateAppearances': 60,
    'baseOnBalls': 4
}

BAYESIAN_MULTIPLIER = 1  # Adjust this multiplier as needed for your model

for x in bayesian:
    bayesian[x] *= BAYESIAN_MULTIPLIER

In [89]:
twothree_df['home_era'] = (twothree_df['home_earnedRuns']  + bayesian['earnedRuns']) / ((twothree_df['home_outs'] + bayesian['outs']) / 27)
twothree_df['away_era'] = (twothree_df['away_earnedRuns'] + bayesian['earnedRuns']) / ((twothree_df['away_outs'] + bayesian['outs']) / 27)
twothree_df['home_whip9'] = (twothree_df['home_sp_hits'] + twothree_df['home_sp_baseOnBalls'] + bayesian['wh']) / ((twothree_df['home_outs'] + bayesian['outs']) / 3)
twothree_df['away_whip9'] = (twothree_df['away_sp_hits'] + twothree_df['away_sp_baseOnBalls'] + bayesian['wh']) / ((twothree_df['away_outs'] + bayesian['outs']) / 3)

for batting_stat in ['hits', 'totalBases', 'homeRuns', 'baseOnBalls']:
    twothree_df[f'home_{batting_stat}'] = sum([(twothree_df[f'home_{i}_{batting_stat}'] + bayesian[batting_stat]) / (twothree_df[f'home_{i}_plateAppearances'] + bayesian['plateAppearances']) for i in range(9)])/9
    twothree_df[f'away_{batting_stat}'] = sum([(twothree_df[f'away_{i}_{batting_stat}'] + bayesian[batting_stat]) / (twothree_df[f'away_{i}_plateAppearances'] + bayesian['plateAppearances']) for i in range(9)])/9


KeyError: 'home_earnedRuns'

### New Approach - Create A Bayesian Based on Preseason Marcel Projections

In [71]:
# Mapping MLB statsapi field names to Baseball Reference/Marcel projection column names
mlb_to_bbref_mapping = {
    "hits": "H",
    "totalBases": "TB", 
    "homeRuns": "HR",
    "plateAppearances": "PA",
    "baseOnBalls": "BB",
    "playername": "Name",
    "earnedRuns": "ER",
    "baseOnBalls": "BB",
    "homeRuns": "HR",
    "innings": "IP"  # Innings pitched * 3
}

MARCEL_REPLACEMENT_BATTING_STATS = {
    "H": 0.20,
    "TB": 0.35,
    "HR": 0.025,
    "BB": 0.06, 
}

MARCEL_REPLACEMENT_PITCHING_STATS = {
    "ER": 0.6,
    "H": 1.0,
    "BB": 0.3,
    "HR": 0.15,
}

In [72]:
RATE_SUFFIX = {
    False: "/pa",
    True: "/ip"
}


def get_marcel_projected_stat(hitter_projections: pd.DataFrame, 
                              playername: pd.Series, 
                              stat: str, 
                              pitcher: bool = False) -> pd.DataFrame:
    """
    Get Marcel projected stat for a given player and statistic.
    
    Parameters:
    - hitter_projections: DataFrame with Marcel projections. Must contain at least
      ['Name', stat] where 'Name' matches playername entries.
    - playername: A pandas Series (or DataFrame column) of player names to look up.
    - stat: Statistic to retrieve (using MLB statsapi / Marcel field names).
    - pitcher: Bool, whether to look up pitcher stats instead of hitter stats (defaults to False).
    
    Returns:
    - DataFrame with ['Name', stat] for each player in playername. If not found, returns NaN.
    """
    
    # Ensure consistent casing/stripping for merges
    hitter_projections = hitter_projections.copy()
    hitter_projections["Name"] = hitter_projections["Name"].str.strip()
    
    players_df = pd.DataFrame({"Name": playername.astype(str).str.strip()})

    
    # Merge on player name
    merged = players_df.merge(
    hitter_projections[["Name", mlb_to_bbref_mapping[stat] + RATE_SUFFIX[pitcher]]],
    on="Name",
    how="left"
)
    
    #the dataset might be missing player name matches, so have a default value in case the player name isn't found
    if merged[mlb_to_bbref_mapping[stat] + RATE_SUFFIX[pitcher]].isnull().any():
        replacement_value = MARCEL_REPLACEMENT_PITCHING_STATS[mlb_to_bbref_mapping[stat]] if pitcher else MARCEL_REPLACEMENT_BATTING_STATS[mlb_to_bbref_mapping[stat]]
        merged[mlb_to_bbref_mapping[stat] + RATE_SUFFIX[pitcher]].fillna(replacement_value, inplace=True)
    
    
    
    return merged


def get_marcel_pitcher_stat(pitcher_projections, playername, stat):
    """
    Get Marcel projected stat for a pitcher.
    
    Parameters:
    - pitcher_projections: DataFrame with Marcel pitcher projections
    - playername: Name of the pitcher to look up
    - stat: Statistic to retrieve
    
    Returns:
    - Projected stat value or None if player not found
    """
    # Handle different name formats and case sensitivity
    player_row = pitcher_projections[pitcher_projections['Name'].str.contains(playername, case=False, na=False)]
    
    if player_row.empty:
        # Try alternative matching by splitting names
        name_parts = playername.split()
        if len(name_parts) >= 2:
            first_name, last_name = name_parts[0], name_parts[-1]
            player_row = pitcher_projections[
                (pitcher_projections['Name'].str.contains(first_name, case=False, na=False)) &
                (pitcher_projections['Name'].str.contains(last_name, case=False, na=False))
            ]
    
    if player_row.empty:
        return None
    
    # Map pitching stats
    pitcher_mapping = {
        "earnedRuns": "ER",
        "hits": "H",
        "baseOnBalls": "BB",
        "homeRuns": "HR",
        "innnings": "IP"  # Innings pitched * 3
    }

    
    
    # Get the Baseball Reference column name
    bbref_stat = pitcher_mapping.get(stat, stat)
    
    if bbref_stat in player_row.columns:
        return player_row.iloc[0][bbref_stat + "/ip"]  # Return per inning stat
    else:
        return None

In [73]:
BAYESIAN_PAS_ADDED = 100
BAYESIAN_IPS_ADDED = 20

BAYESIAN_ADDED = {
    False: BAYESIAN_PAS_ADDED, 
    True: BAYESIAN_IPS_ADDED
}


def integrate_bayesian(df, player_number, stat, hitter_projections, home=True):
    """
    Integrate Bayesian priors with individual player stats
    
    Parameters:
    - df: DataFrame with game data
    - player_number: Player position (0-8 for batting order)
    - stat: Statistic to integrate ('hits', 'totalBases', 'homeRuns', 'baseOnBalls')
    - home: Boolean, True for home team, False for away team
    """
    home_str = 'home' if home else 'away'
    pitcher = True if player_number == "sp" else False
    print("testing", player_number, stat, pitcher, RATE_SUFFIX[pitcher])

    original_col = f'{home_str}_{player_number}_{stat}'
    pa_col = f'{home_str}_{player_number}_plateAppearances'
    ip_col = f'{home_str}_sp_outs'
    bayesian_col = original_col + "_bayesian"
    marcel_col = original_col + "_marcel_projection"
    playername_dfcol = df[f'{home_str}_{player_number}_playername']

    df[marcel_col] = get_marcel_projected_stat(hitter_projections, playername_dfcol, stat, pitcher=pitcher)[mlb_to_bbref_mapping[stat] + RATE_SUFFIX[pitcher]] * BAYESIAN_ADDED[pitcher]

    # Apply Bayesian integration: (actual + prior) / (PA + prior_PA)
    if not pitcher:
        df[bayesian_col] = (df[original_col] + df[marcel_col]) / ((df[pa_col]) + BAYESIAN_ADDED[pitcher])
    else:
        df[bayesian_col] = (df[original_col] + df[marcel_col]) / ((df[ip_col] // 3) + BAYESIAN_ADDED[pitcher])
    
    return df

def integrate_all_bayesians(df, hprojs, pprojs):
    """
    Apply Bayesian integration to all players and relevant stats
    """
    relevant_stats = ['hits', 'totalBases', 'homeRuns', 'baseOnBalls']
    
    # Apply to all players (0-8) for both home and away teams
    for player_number in range(9):  # Players 0-8 in batting order
        for stat in relevant_stats:
            # Integrate for home team
            df = integrate_bayesian(df, player_number, stat, hprojs, home=True)
            # Integrate for away team  
            df = integrate_bayesian(df, player_number, stat, hprojs, home=False)

    # pitchers
    for pitching_stat in ["earnedRuns", "hits", "baseOnBalls", "homeRuns"]:
        for home_bool in [True, False]:
            df = integrate_bayesian(df, "sp", pitching_stat, pprojs, home=home_bool)
    
    return df

# Example usage:
twothree_df = integrate_all_bayesians(twothree_df, hitter_projections, pitcher_projections)

testing 0 hits False /pa
testing 0 hits False /pa
testing 0 totalBases False /pa
testing 0 totalBases False /pa
testing 0 homeRuns False /pa
testing 0 homeRuns False /pa
testing 0 baseOnBalls False /pa
testing 0 baseOnBalls False /pa
testing 1 hits False /pa
testing 1 hits False /pa
testing 1 totalBases False /pa
testing 1 totalBases False /pa
testing 1 homeRuns False /pa
testing 1 homeRuns False /pa
testing 1 baseOnBalls False /pa
testing 1 baseOnBalls False /pa
testing 2 hits False /pa
testing 2 hits False /pa
testing 2 totalBases False /pa
testing 2 totalBases False /pa
testing 2 homeRuns False /pa
testing 2 homeRuns False /pa
testing 2 baseOnBalls False /pa
testing 2 baseOnBalls False /pa
testing 3 hits False /pa
testing 3 hits False /pa
testing 3 totalBases False /pa
testing 3 totalBases False /pa
testing 3 homeRuns False /pa
testing 3 homeRuns False /pa
testing 3 baseOnBalls False /pa
testing 3 baseOnBalls False /pa
testing 4 hits False /pa
testing 4 hits False /pa
testing 4 tota

In [74]:
def aggregate_bayesian_team_stats(df):
    """
    Aggregate individual Bayesian-adjusted player stats into team-level totals
    Equivalent to the old approach summation but using Marcel-based Bayesian stats
    """
    relevant_stats = ['hits', 'totalBases', 'homeRuns', 'baseOnBalls']
    
    for batting_stat in relevant_stats:
        # Sum Bayesian-adjusted stats for home team (players 0-8)
        df[f'home_{batting_stat}'] = sum([df[f'home_{i}_{batting_stat}_bayesian'] for i in range(9) if f'home_{i}_{batting_stat}_bayesian' in df.columns])/9
        
        # Sum Bayesian-adjusted stats for away team (players 0-8)  
        df[f'away_{batting_stat}'] = sum([df[f'away_{i}_{batting_stat}_bayesian'] for i in range(9) if f'away_{i}_{batting_stat}_bayesian' in df.columns])/9

    print(df.columns)
    #TODO: PICTHING STATS
    df['home_era'] = df['home_sp_earnedRuns_bayesian']
    df['away_era'] = df['away_sp_earnedRuns_bayesian']
    df['home_whip9'] = df['home_sp_hits_bayesian'] + df['home_sp_baseOnBalls_bayesian']
    df['away_whip9'] = df['away_sp_hits_bayesian'] + df['away_sp_baseOnBalls_bayesian']

    return df

twothree_df = aggregate_bayesian_team_stats(twothree_df)

Index(['gamePk', 'home_wins', 'home_losses', 'home_winpercentage', 'home_elo',
       'away_elo', 'away_wins', 'away_losses', 'away_winpercentage',
       'away_runs',
       ...
       'away_era', 'home_whip9', 'away_whip9', 'era_diff', 'wp_diff',
       'elo_diff', 'whip9_diff', 'obp_diff', 'slg_diff', 'HRR_diff'],
      dtype='object', length=310)


In [75]:

# Get rid of NaN values (note: this is when a pitcher makes his season debut)
twothree_df = twothree_df.dropna(subset=['home_era', 'away_era'])
twothree_df.head()  # Display the first few rows of the DataFrame

twothree_df['era_diff'] = twothree_df['home_era'] - twothree_df['away_era']
twothree_df['wp_diff'] = twothree_df['home_winpercentage'] - twothree_df['away_winpercentage']
twothree_df['elo_diff'] = twothree_df['home_elo'] - twothree_df['away_elo']
twothree_df['whip9_diff'] = twothree_df['home_whip9'] - twothree_df['away_whip9']
twothree_df['obp_diff'] = twothree_df['home_hits'] + twothree_df['home_baseOnBalls'] - twothree_df['away_hits'] - twothree_df['away_baseOnBalls']
twothree_df['slg_diff'] = twothree_df['home_totalBases'] - twothree_df['away_totalBases']
twothree_df['HRR_diff'] = twothree_df['home_homeRuns'] - twothree_df['away_homeRuns']
twothree_df.tail()  # Display the updated DataFrame with new features

,gamePk,home_wins,home_losses,home_winpercentage,home_elo,away_elo,away_wins,away_losses,away_winpercentage,away_runs,home_win,home_sp_earnedRuns,home_sp_hits,home_sp_baseOnBalls,home_sp_homeRuns,home_sp_outs,home_sp_playername,away_sp_earnedRuns,away_sp_hits,away_sp_baseOnBalls,away_sp_homeRuns,away_sp_outs,away_sp_playername,home_0_hits,home_1_hits,home_2_hits,home_3_hits,home_4_hits,home_5_hits,home_6_hits,home_7_hits,home_8_hits,home_0_totalBases,home_1_totalBases,home_2_totalBases,home_3_totalBases,home_4_totalBases,home_5_totalBases,home_6_totalBases,home_7_totalBases,home_8_totalBases,home_0_homeRuns,home_1_homeRuns,home_2_homeRuns,home_3_homeRuns,home_4_homeRuns,home_5_homeRuns,home_6_homeRuns,home_7_homeRuns,home_8_homeRuns,home_0_plateAppearances,home_1_plateAppearances,home_2_plateAppearances,home_3_plateAppearances,home_4_plateAppearances,home_5_plateAppearances,home_6_plateAppearances,home_7_plateAppearances,home_8_plateAppearances,home_0_baseOnBalls,home_1_baseOnBalls,home_2_baseOnBalls,home_3_baseOnBalls,home_4_baseOnBalls,home_5_baseOnBalls,home_6_baseOnBalls,home_7_baseOnBalls,home_8_baseOnBalls,home_0_playername,home_1_playername,home_2_playername,home_3_playername,home_4_playername,home_5_playername,home_6_playername,home_7_playername,home_8_playername,away_0_hits,away_1_hits,away_2_hits,away_3_hits,away_4_hits,away_5_hits,away_6_hits,away_7_hits,away_8_hits,away_0_totalBases,away_1_totalBases,away_2_totalBases,away_3_totalBases,away_4_totalBases,away_5_totalBases,away_6_totalBases,away_7_totalBases,away_8_totalBases,away_0_homeRuns,away_1_homeRuns,away_2_homeRuns,away_3_homeRuns,away_4_homeRuns,away_5_homeRuns,away_6_homeRuns,away_7_homeRuns,away_8_homeRuns,away_0_plateAppearances,away_1_plateAppearances,away_2_plateAppearances,away_3_plateAppearances,away_4_plateAppearances,away_5_plateAppearances,away_6_plateAppearances,away_7_plateAppearances,away_8_plateAppearances,away_0_baseOnBalls,away_1_baseOnBalls,away_2_baseOnBalls,away_3_baseOnBalls,away_4_baseOnBalls,away_5_baseOnBalls,away_6_baseOnBalls,away_7_baseOnBalls,away_8_baseOnBalls,away_0_playername,away_1_playername,away_2_playername,away_3_playername,away_4_playername,away_5_playername,away_6_playername,away_7_playername,away_8_playername,home_0_hits_marcel_projection,home_0_hits_bayesian,away_0_hits_marcel_projection,away_0_hits_bayesian,home_0_totalBases_marcel_projection,home_0_totalBases_bayesian,away_0_totalBases_marcel_projection,away_0_totalBases_bayesian,home_0_homeRuns_marcel_projection,home_0_homeRuns_bayesian,away_0_homeRuns_marcel_projection,away_0_homeRuns_bayesian,home_0_baseOnBalls_marcel_projection,home_0_baseOnBalls_bayesian,away_0_baseOnBalls_marcel_projection,away_0_baseOnBalls_bayesian,home_1_hits_marcel_projection,home_1_hits_bayesian,away_1_hits_marcel_projection,away_1_hits_bayesian,home_1_totalBases_marcel_projection,home_1_totalBases_bayesian,away_1_totalBases_marcel_projection,away_1_totalBases_bayesian,home_1_homeRuns_marcel_projection,home_1_homeRuns_bayesian,away_1_homeRuns_marcel_projection,away_1_homeRuns_bayesian,home_1_baseOnBalls_marcel_projection,home_1_baseOnBalls_bayesian,away_1_baseOnBalls_marcel_projection,away_1_baseOnBalls_bayesian,home_2_hits_marcel_projection,home_2_hits_bayesian,away_2_hits_marcel_projection,away_2_hits_bayesian,home_2_totalBases_marcel_projection,home_2_totalBases_bayesian,away_2_totalBases_marcel_projection,away_2_totalBases_bayesian,home_2_homeRuns_marcel_projection,home_2_homeRuns_bayesian,away_2_homeRuns_marcel_projection,away_2_homeRuns_bayesian,home_2_baseOnBalls_marcel_projection,home_2_baseOnBalls_bayesian,away_2_baseOnBalls_marcel_projection,away_2_baseOnBalls_bayesian,home_3_hits_marcel_projection,home_3_hits_bayesian,away_3_hits_marcel_projection,away_3_hits_bayesian,home_3_totalBases_marcel_projection,home_3_totalBases_bayesian,away_3_totalBases_marcel_projection,away_3_totalBases_bayesian,home_3_homeRuns_marcel_projection,home_3_homeRuns_bayesian,away_3_homeRuns_marcel_project

In [76]:
# Testing the Bayesian results
# get the last row
test_row = twothree_df.iloc[-1]
print(test_row['away_0_playername'], test_row['away_0_hits'], test_row['away_0_plateAppearances'],test_row['away_0_hits_bayesian'], test_row['away_0_hits_marcel_projection'])

Jon Berti 111 408 0.25888350494649703 20.51282051282051


In [77]:
twothree_df

,gamePk,home_wins,home_losses,home_winpercentage,home_elo,away_elo,away_wins,away_losses,away_winpercentage,away_runs,home_win,home_sp_earnedRuns,home_sp_hits,home_sp_baseOnBalls,home_sp_homeRuns,home_sp_outs,home_sp_playername,away_sp_earnedRuns,away_sp_hits,away_sp_baseOnBalls,away_sp_homeRuns,away_sp_outs,away_sp_playername,home_0_hits,home_1_hits,home_2_hits,home_3_hits,home_4_hits,home_5_hits,home_6_hits,home_7_hits,home_8_hits,home_0_totalBases,home_1_totalBases,home_2_totalBases,home_3_totalBases,home_4_totalBases,home_5_totalBases,home_6_totalBases,home_7_totalBases,home_8_totalBases,home_0_homeRuns,home_1_homeRuns,home_2_homeRuns,home_3_homeRuns,home_4_homeRuns,home_5_homeRuns,home_6_homeRuns,home_7_homeRuns,home_8_homeRuns,home_0_plateAppearances,home_1_plateAppearances,home_2_plateAppearances,home_3_plateAppearances,home_4_plateAppearances,home_5_plateAppearances,home_6_plateAppearances,home_7_plateAppearances,home_8_plateAppearances,home_0_baseOnBalls,home_1_baseOnBalls,home_2_baseOnBalls,home_3_baseOnBalls,home_4_baseOnBalls,home_5_baseOnBalls,home_6_baseOnBalls,home_7_baseOnBalls,home_8_baseOnBalls,home_0_playername,home_1_playername,home_2_playername,home_3_playername,home_4_playername,home_5_playername,home_6_playername,home_7_playername,home_8_playername,away_0_hits,away_1_hits,away_2_hits,away_3_hits,away_4_hits,away_5_hits,away_6_hits,away_7_hits,away_8_hits,away_0_totalBases,away_1_totalBases,away_2_totalBases,away_3_totalBases,away_4_totalBases,away_5_totalBases,away_6_totalBases,away_7_totalBases,away_8_totalBases,away_0_homeRuns,away_1_homeRuns,away_2_homeRuns,away_3_homeRuns,away_4_homeRuns,away_5_homeRuns,away_6_homeRuns,away_7_homeRuns,away_8_homeRuns,away_0_plateAppearances,away_1_plateAppearances,away_2_plateAppearances,away_3_plateAppearances,away_4_plateAppearances,away_5_plateAppearances,away_6_plateAppearances,away_7_plateAppearances,away_8_plateAppearances,away_0_baseOnBalls,away_1_baseOnBalls,away_2_baseOnBalls,away_3_baseOnBalls,away_4_baseOnBalls,away_5_baseOnBalls,away_6_baseOnBalls,away_7_baseOnBalls,away_8_baseOnBalls,away_0_playername,away_1_playername,away_2_playername,away_3_playername,away_4_playername,away_5_playername,away_6_playername,away_7_playername,away_8_playername,home_0_hits_marcel_projection,home_0_hits_bayesian,away_0_hits_marcel_projection,away_0_hits_bayesian,home_0_totalBases_marcel_projection,home_0_totalBases_bayesian,away_0_totalBases_marcel_projection,away_0_totalBases_bayesian,home_0_homeRuns_marcel_projection,home_0_homeRuns_bayesian,away_0_homeRuns_marcel_projection,away_0_homeRuns_bayesian,home_0_baseOnBalls_marcel_projection,home_0_baseOnBalls_bayesian,away_0_baseOnBalls_marcel_projection,away_0_baseOnBalls_bayesian,home_1_hits_marcel_projection,home_1_hits_bayesian,away_1_hits_marcel_projection,away_1_hits_bayesian,home_1_totalBases_marcel_projection,home_1_totalBases_bayesian,away_1_totalBases_marcel_projection,away_1_totalBases_bayesian,home_1_homeRuns_marcel_projection,home_1_homeRuns_bayesian,away_1_homeRuns_marcel_projection,away_1_homeRuns_bayesian,home_1_baseOnBalls_marcel_projection,home_1_baseOnBalls_bayesian,away_1_baseOnBalls_marcel_projection,away_1_baseOnBalls_bayesian,home_2_hits_marcel_projection,home_2_hits_bayesian,away_2_hits_marcel_projection,away_2_hits_bayesian,home_2_totalBases_marcel_projection,home_2_totalBases_bayesian,away_2_totalBases_marcel_projection,away_2_totalBases_bayesian,home_2_homeRuns_marcel_projection,home_2_homeRuns_bayesian,away_2_homeRuns_marcel_projection,away_2_homeRuns_bayesian,home_2_baseOnBalls_marcel_projection,home_2_baseOnBalls_bayesian,away_2_baseOnBalls_marcel_projection,away_2_baseOnBalls_bayesian,home_3_hits_marcel_projection,home_3_hits_bayesian,away_3_hits_marcel_projection,away_3_hits_bayesian,home_3_totalBases_marcel_projection,home_3_totalBases_bayesian,away_3_totalBases_marcel_projection,away_3_totalBases_bayesian,home_3_homeRuns_marcel_projection,home_3_homeRuns_bayesian,away_3_homeRuns_marcel_project

In [78]:
print(twothree_df['home_era'].mean())
print(twothree_df['away_era'].mean())
print(twothree_df['home_whip9'].mean())
print(twothree_df['away_whip9'].mean())

0.4735441918073316
0.4702200258921263
1.282375229855711
1.2794611727735619


## Model Training and Evaluation

Now let's train regression models to predict the probability of home team wins (which can be converted to moneylines).

In [83]:
feature_cols = [
    'era_diff','whip9_diff', 'obp_diff', 'slg_diff', 'wp_diff', 'elo_diff', 'HRR_diff',   
]

#import boossted_stump
from sklearn.ensemble import GradientBoostingRegressor

#import K nearest Neighbors
from sklearn.neighbors import KNeighborsRegressor

def train_moneyline_models(features_df):
    """Train models to predict home team win probability"""
    if features_df is None or len(features_df) < 10:
        print("Insufficient data for training")
        return None, None
    
    # Select feature columns (exclude metadata and target)
    X = features_df[feature_cols]
    y = features_df['home_win']


    print(f"Training on {len(X)} games with {len(feature_cols)} features")
    print(f"Home team win rate: {y.mean():.3f}")
    
    # Normalize features
    # for col in X.columns:
    #     print(f"Normalizing feature: {col}")
    #     print(f"  Mean: {X[col].mean():.4f}, Std: {X[col].std():.4f}")
    X = (X - X.mean()) / X.std()

    # Split data
    if len(X) > 20:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=random.randint(1, 10000))
    else:
        # Use all data for training if dataset is small
        X_train, X_test, y_train, y_test = X, X, y, y
        print("Small dataset - using all data for both training and testing")
    
    # Train models
    models = {}

    # KNN Regressor
    knn_model = KNeighborsRegressor(n_neighbors=50)
    knn_model.fit(X_train, y_train)
    models['KNN Regressor'] = knn_model

    
    # Logistic Regression
    lgr_model = LogisticRegression()
    lgr_model.fit(X_train, y_train)
    models['Logistic Regression'] = lgr_model
    
    # Random Forest
    rf_model = RandomForestRegressor(n_estimators=100, random_state=random.randint(1, 10000))
    rf_model.fit(X_train, y_train)
    models['Random Forest'] = rf_model
    
    # Evaluate models
    results = {}
    for name, model in models.items():
        if hasattr(model, 'predict_proba'):
            # For classification models
            y_pred = model.predict_proba(X_test)[:, 1]
        else:
            # For regression models
            y_pred = model.predict(X_test)
        # Clip predictions to [0, 1] range for probability
        y_pred = np.clip(y_pred, 0, 1)
        
        mse = mean_squared_error(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        
        # Classification accuracy (using 0.5 threshold)
        y_pred_class = (y_pred > 0.5).astype(int)
        accuracy = (y_pred_class == y_test).mean()
        
        results[name] = {
            'MSE': mse,
            'MAE': mae,
            'R²': r2,
            'Accuracy': accuracy,
            'predictions': y_pred
        }
        
        print(f"\n{name} Results:")
        print(f"  MSE: {mse:.4f}")
        print(f"  MAE: {mae:.4f}")
        print(f"  R²: {r2:.4f}")
        print(f"  Accuracy: {accuracy:.4f}")
    
    return models, results

# Train models
all_models, all_results = list(), list()
for i in range(10):
    print(f"Training iteration {i+1}")
    models, results = train_moneyline_models(twothree_df)
    all_models.append(models)
    all_results.append(results)

print("After training 10 iterations:")
#Print average results
avg_results = {name: {metric: np.mean([result[name][metric] for result in all_results]) for metric in all_results[0][name]} for name in all_results[0]}
for name, metrics in avg_results.items():
    print(f"\n{name} Average Results:")
    for metric, value in metrics.items():
        if metric == 'predictions':
            continue
        print(f"  {metric}: {value:.4f}")



Training iteration 1
Training on 2476 games with 7 features
Home team win rate: 0.520

KNN Regressor Results:
  MSE: 0.2421
  MAE: 0.4821
  R²: 0.0249
  Accuracy: 0.5760

Logistic Regression Results:
  MSE: 0.2414
  MAE: 0.4852
  R²: 0.0278
  Accuracy: 0.5814

Random Forest Results:
  MSE: 0.2422
  MAE: 0.4663
  R²: 0.0248
  Accuracy: 0.5976
Training iteration 2
Training on 2476 games with 7 features
Home team win rate: 0.520

KNN Regressor Results:
  MSE: 0.2450
  MAE: 0.4846
  R²: 0.0190
  Accuracy: 0.5437

Logistic Regression Results:
  MSE: 0.2463
  MAE: 0.4887
  R²: 0.0139
  Accuracy: 0.5518

Random Forest Results:
  MSE: 0.2427
  MAE: 0.4665
  R²: 0.0282
  Accuracy: 0.5720
Training iteration 3
Training on 2476 games with 7 features
Home team win rate: 0.520

KNN Regressor Results:
  MSE: 0.2469
  MAE: 0.4859
  R²: 0.0028
  Accuracy: 0.5478

Logistic Regression Results:
  MSE: 0.2453
  MAE: 0.4880
  R²: 0.0093
  Accuracy: 0.5424

Random Forest Results:
  MSE: 0.2507
  MAE: 0.4765


In [80]:
feature_cols = [
    'era', 'whip9', 'avg', 'slg', 'HRR',
]

def train_elo_model(feature_df):
    elo_df = pd.DataFrame()

    home_dict = {col: feature_df[f'home_pregame_{col}'] for col in feature_cols}
    home_dict['win'] = feature_df['home_win']

    away_dict = {col: feature_df[f'away_pregame_{col}'] for col in feature_cols}
    away_dict['win'] = 1 - feature_df['home_win']  # Invert home win to get away win

    elo_df = pd.concat([pd.DataFrame(home_dict), pd.DataFrame(away_dict)], ignore_index=True)
    print(f"Training Elo model on {len(elo_df)} games with {len(feature_cols)} features")

In [81]:
# Show each coefficients for the Linear Regression model
print("\nLinear Regression Coefficients:")
for feature, coef in zip(feature_cols, models['Linear Regression'].coef_):
    print(f"{feature}: {coef:.4f}")
    print("appx imporance:", abs(coef) / sum(abs(models['Linear Regression'].coef_)))
print("LinearRegression constant:", models['Linear Regression'].intercept_)


# Show each coefficients for the Logistic Regression model
print("\nLogistic Regression Coefficients:")
for feature, coef in zip(feature_cols, models['Logistic Regression'].coef_[0]):
    print(f"{feature}: {coef:.4f}")
print("LogisticRegression constant:", models['Logistic Regression'].intercept_[0])
print("Expected Home Win Probability based on Intercept:", 
      1 / (1 + np.exp(-models['Logistic Regression'].intercept_[0])))

# Plotting feature importance for Random Forest model
if 'Random Forest' in models:
    rf_importances = models['Random Forest'].feature_importances_
    feature_importance_df = pd.DataFrame({
        'Feature': feature_cols,
        'Importance': rf_importances
    }).sort_values(by='Importance', ascending=False)

    plt.figure(figsize=(10, 6))
    sns.barplot(x='Importance', y='Feature', data=feature_importance_df)
    plt.title('Random Forest Feature Importance')
    plt.xlabel('Importance')
    plt.ylabel('Feature')
    plt.show()




Linear Regression Coefficients:


KeyError: 'Linear Regression'

## Moneyline Conversion and Predictions

Convert win probabilities to moneylines and make predictions for upcoming games.

In [ ]:
def probability_to_moneyline(prob):
    """Convert win probability to American moneyline odds"""
    if prob <= 0 or prob >= 1:
        return None
    
    if prob > 0.5:
        # Favorite (negative odds)
        return -int(prob / (1 - prob) * 100)
    else:
        # Underdog (positive odds)
        return int((1 - prob) / prob * 100)

def moneyline_to_probability(moneyline):
    """Convert American moneyline odds to win probability"""
    if moneyline > 0:
        return 100 / (moneyline + 100)
    else:
        return abs(moneyline) / (abs(moneyline) + 100)

def get_todays_games():
    """Get today's scheduled games for predictions"""
    try:
        today = datetime.now().strftime('%Y-%m-%d')
        schedule = mlbstatsapi.schedule(date=today)
        
        todays_games = []
        for game in schedule:
            if game['status']['statusCode'] in ['S', 'P']:  # Scheduled or Pre-game
                game_data = {
                    'game_id': game['gamePk'],
                    'date': today,
                    'home_team_id': game['teams']['home']['team']['id'],
                    'home_team_name': game['teams']['home']['team']['name'],
                    'away_team_id': game['teams']['away']['team']['id'],
                    'away_team_name': game['teams']['away']['team']['name'],
                    'game_time': game.get('gameDate', 'TBD')
                }
                todays_games.append(game_data)
        
        return pd.DataFrame(todays_games)
    except Exception as e:
        print(f"Error getting today's games: {e}")
        return None

# Get today's games and make predictions
if 'models' in locals() and models is not None and 'standings_df' in locals():
    todays_games = get_todays_games()
    
    if todays_games is not None and len(todays_games) > 0:
        print(f"Found {len(todays_games)} games scheduled for today:")
        
        # Create features for today's games
        todays_features = create_features_for_games(todays_games, standings_df)
        
        if todays_features is not None:
            feature_cols = [
                'home_win_pct', 'away_win_pct', 'win_pct_diff',
                'home_run_diff', 'away_run_diff', 'run_diff_advantage',
                'home_field_advantage', 'home_team_era', 'away_team_era', 'era_advantage'
            ]
            
            X_today = todays_features[feature_cols]
            
            # Make predictions with best model (Random Forest)
            if 'Random Forest' in models:
                predictions = models['Random Forest'].predict(X_today)
                predictions = np.clip(predictions, 0.01, 0.99)  # Avoid extreme probabilities
                
                print("\n=== TODAY'S PREDICTIONS ===")
                for i, (_, game) in enumerate(todays_features.iterrows()):
                    home_prob = predictions[i]
                    away_prob = 1 - home_prob
                    
                    home_ml = probability_to_moneyline(home_prob)
                    away_ml = probability_to_moneyline(away_prob)
                    
                    print(f"\nGame: {game['away_team_name']} @ {game['home_team_name']}")
                    print(f"  Home Win Probability: {home_prob:.3f} ({home_ml:+d})")
                    print(f"  Away Win Probability: {away_prob:.3f} ({away_ml:+d})")
                    
                    # Determine recommendation
                    if home_prob > 0.55:
                        print(f"  Recommendation: Bet {game['home_team_name']} (Home)")
                    elif away_prob > 0.55:
                        print(f"  Recommendation: Bet {game['away_team_name']} (Away)")
                    else:
                        print(f"  Recommendation: No strong edge detected")
            else:
                print("Random Forest model not available for predictions")
        else:
            print("Could not create features for today's games")
    else:
        print("No games scheduled for today or could not retrieve schedule")
else:
    print("Models not trained - cannot make predictions")

In [ ]:
models, results = train_moneyline_models(twothree_df)